In [87]:
# Own Libraries
from utils.data import load_data
from utils.metrics import prec_recall, iou_score, f1_dice
from utils.similarity import Similarity
from utils.image_processing import image_to_windows, get_3d_norm_histogram, calculate_histograms
# 3rd Party Libraries
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import scipy.ndimage

In [47]:
db, db_files = load_data("./data/BBDD/",".jpg", desc = "Loading BBDD Data...")
qsd1_w2, qsd1_w2_files = load_data("./data/qsd1_w2/",".jpg", desc = "Loading qsd1_w2 Data...")

Loading qsd1_w2 Data...:   0%|                                                                  | 0/30 [00:00<?, ?it/s]

./data/BBDD/ read: 287 images


Loading qsd1_w2 Data...: 100%|█████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 81.61it/s]


./data/qsd1_w2/ read: 30 images


In [229]:
with open("./data/qsd1_w2/text_boxes.pkl", "rb") as file:
    gt_boxes = pickle.load(file)
    print(gt_boxes)


[[[array([101,   6]), array([101, 107]), array([571, 107]), array([571,   6])]], [[array([68, 18]), array([68, 97]), array([384,  97]), array([384,  18])]], [[array([ 506, 1797]), array([ 506, 1882]), array([1106, 1882]), array([1106, 1797])]], [[array([576, 106]), array([576, 235]), array([1176,  235]), array([1176,  106])]], [[array([ 90, 523]), array([ 90, 556]), array([509, 556]), array([509, 523])]], [[array([54, 22]), array([54, 55]), array([307,  55]), array([307,  22])]], [[array([47,  1]), array([ 47, 123]), array([265, 123]), array([265,   1])]], [[array([526, 149]), array([526, 212]), array([1126,  212]), array([1126,  149])]], [[array([ 44, 350]), array([ 44, 379]), array([250, 379]), array([250, 350])]], [[array([ 38, 269]), array([ 38, 309]), array([216, 309]), array([216, 269])]], [[array([101, 513]), array([101, 556]), array([574, 556]), array([574, 513])]], [[array([31, 10]), array([31, 41]), array([179,  41]), array([179,  10])]], [[array([49, 17]), array([49, 43]), a

In [291]:
def text_detection(image:np.ndarray):
    
    #Transform to grayscale
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Defining the kernels to use
    filterSize =(12, 12)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, 
                                   filterSize)
    
    #Applying tophat and blackhat morph operators
    tophat_img = cv2.morphologyEx(img, 
                              cv2.MORPH_TOPHAT,
                              kernel)
    blackhat_img = cv2.morphologyEx(img, 
                              cv2.MORPH_BLACKHAT,
                              kernel)
    #Defining the kernels to use
    filterSize =(3, 3)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, 
                                   filterSize)
    #Adding the results of tophat and blackhat morph operators and thresholding 
    #to get the letters that have the most intensity
    mean = tophat_img/2 +blackhat_img/2
    hat = np.uint8(mean>70)
    
    #Opening filter to remove little artifacts
    opening = cv2.morphologyEx(hat, cv2.MORPH_OPEN, kernel,iterations = 1)

    #Defining the kernels to use
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (80,18 ))

    # Applying dilation on the threshold image
    dilation = cv2.dilate(opening, rect_kernel, iterations = 1)

    # Finding contours
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_NONE)
    
    bounded = np.zeros((img.shape))
    area = 0
    #Loop to get the contour with the greatest area
    for cnt in contours:
        area_i = cv2.contourArea(cnt)
        if area_i >= area:
            area = area_i
            x, y, w, h = cv2.boundingRect(cnt)
    coordinates = list()
    coordinates.append([[x,y],[x,y+h],[x+w,y+h],[x+w,y]])
    
    
    return coordinates
    

In [298]:
iou=0
for img, gt_coord in zip(qsd1_w2,gt_boxes):
    detected_coord = text_detection(img)
    gt_mask = np.zeros((img.shape[0],img.shape[1]))
    detected_mask = np.zeros((img.shape[0],img.shape[1]))
    top_left = (gt_coord[0][0][0], gt_coord[0][0][1])
    bottom_right = (gt_coord[0][2][0], gt_coord[0][2][1])
    rect = cv2.rectangle(gt_mask, top_left, bottom_right, 255, -1)
    top_left = (detected_coord[0][0][0], detected_coord[0][0][1])
    bottom_right = (detected_coord[0][2][0], detected_coord[0][2][1])
    rect = cv2.rectangle(detected_mask, top_left, bottom_right, 255, -1)
    
    iou_i = iou_score(gt_mask,detected_mask)
    iou += iou_i
print(iou/len(qsd1_w2))
print(len(qsd1_w2))

0.6015927463526078
30
